# **CIÊNCIA DE DADOS**

**Bibliotecas principais**

In [90]:
# Importando as bibliotecas necessárias
import pandas as pd  # Manipulação de dados
import numpy as np  # Operações matemáticas e arrays
import seaborn as sns  # Visualização de dados
import matplotlib.pyplot as plt  # Gráficos
import unicodedata  # Remoção de acentos
import re  # Expressões regulares para validação de dados
import logging  # Registro de logs para facilitar o acompanhamento do processo
from sklearn.impute import SimpleImputer  # Preenchimento de valores ausentes

# A função do logger é registrar o que ocorre em cada etapa do código,
# tornando mais fácil identificar problemas e acompanhar o andamento
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()  # Criando um logger para a aplicação

In [91]:
# Carrega o Dataset
caminho = "/kaggle/input/trabalhocienciadedados/vendas_modificado(in).csv"
df = pd.read_csv(caminho)

# Mostra as Infos Iniciais
print("Documento lido com sucesso!")
print("Documento recebido: \n")
print(df.head(3))

Documento lido com sucesso!
Documento recebido: 

   id_da_compra        data      hora                    cliente  \
0         13679   3/20/2021  23:35:51          LUCAS Araujo Kuhn   
1         28070  10/30/2020   9:00:53       MICAEL SOUZA RONCETE   
2         47484    6/9/2021  15:30:28  FELIPE AUGUSTO NERY SILVA   

            produto     valor  quantidade   total                status  \
0  Queijo Mussarela  R$ 16,87          13  239.31  Pagamento Confirmado   
1   Molho de Tomate   R$ 3,25           3    9.75  Pagamento Confirmado   
2      Água Mineral   R$ 1,63           9   36.67  Pagamento Confirmado   

      cidade estado    pais        cep  frete               pagamento  \
0    Niterói     RJ  Brasil  24000-000   20.0       Cartão de Crédito   
1    Mariana     MG  Brasil  35420-000    0.0                     Pix   
2  Cabo Frio     RJ  Brasil  28900-000   22.0  Transferência Bancária   

                             vendedor         marca  
0             SAMUEL HENRIQUE

In [92]:
# Função para remover acentos e padronizar texto
def padronizar_texto(texto):
    if pd.isna(texto): return texto
    texto = unicodedata.normalize('NFKD', texto)
    texto = texto.encode('ASCII', 'ignore').decode('utf-8')  # remove acentos
    texto = re.sub(r'[^a-zA-Z0-9_ ]', '', texto)  # remove caracteres especiais
    texto = texto.strip().lower().replace(" ", "_")
    return texto

# Aplica a função nas colunas desejadas
colunas_texto = ['cliente', 'cidade', 'produto', 'status']

for col in colunas_texto:
    if col in df.columns:
        print(f"\n{col.upper()}")
        print("Antes:", list(df[col].head(3)))
        df[col] = df[col].astype(str).apply(padronizar_texto)
        print("Depois:", list(df[col].head(3)))


CLIENTE
Antes: ['LUCAS Araujo Kuhn', 'MICAEL SOUZA RONCETE', 'FELIPE AUGUSTO NERY SILVA']
Depois: ['lucas_araujo_kuhn', 'micael_souza_roncete', 'felipe_augusto_nery_silva']

CIDADE
Antes: ['Niterói', 'Mariana', 'Cabo Frio']
Depois: ['niteroi', 'mariana', 'cabo_frio']

PRODUTO
Antes: ['Queijo Mussarela', 'Molho de Tomate', 'Água Mineral']
Depois: ['queijo_mussarela', 'molho_de_tomate', 'agua_mineral']

STATUS
Antes: ['Pagamento Confirmado', 'Pagamento Confirmado', 'Pagamento Confirmado']
Depois: ['pagamento_confirmado', 'pagamento_confirmado', 'pagamento_confirmado']


In [93]:
# Funções de limpeza
def limpar_valor(valor_str):
    match = re.search(r'\d+,\d{1,2}', valor_str)
    return float(match.group(0).replace(',', '.')) if match else None

def limpar_frete(valor_str):
    try:
        val = float(valor_str)
        return val if 0.1 <= val <= 999.99 else None
    except ValueError:
        return None

def limpar_quantidade(valor_str):
    try:
        val = int(float(valor_str))
        return val if 1 <= val <= 2000 else None
    except ValueError:
        return None

# Mapeamento de tipo → função
FUNCOES_LIMPEZA = {
    'valor': limpar_valor,
    'frete': limpar_frete,
    'quantidade': limpar_quantidade,
}

# Função principal de limpeza
def limpar_coluna(df, coluna, tipo_coluna):
    if coluna not in df.columns:
        return df

    func = FUNCOES_LIMPEZA.get(tipo_coluna)
    if not func:
        return df

    df['_corrigido'] = df[coluna].astype(str).str.strip().map(func)

    df_validos = df[df['_corrigido'].notna()].copy()
    df_invalidos = df[df['_corrigido'].isna()]

    df_validos[coluna] = df_validos['_corrigido']
    df_validos.drop(columns=['_corrigido'], inplace=True)

    return df_validos

# Aplicação em massa
for col, tipo in FUNCOES_LIMPEZA.items():
    df = limpar_coluna(df, col, col)

# Mostrar resultado final
print("\nResultado final do DataFrame limpo:\n")
print(df.head())
print(f"\nTotal de registros após limpeza: {len(df)}\n")


Resultado final do DataFrame limpo:

   id_da_compra       data      hora                         cliente  \
0         13679  3/20/2021  23:35:51               lucas_araujo_kuhn   
2         47484   6/9/2021  15:30:28       felipe_augusto_nery_silva   
3         20809   6/4/2022   8:41:23             levi_ribeiro_amorim   
5         38623  2/19/2018  17:32:01     lucas_antonio_de_souza_neto   
6         46627  4/21/2018  22:08:30  guilherme_ivair_de_assis_setta   

            produto  valor  quantidade   total                status  \
0  queijo_mussarela  16.87          13  239.31  pagamento_confirmado   
2      agua_mineral   1.63           9   36.67  pagamento_confirmado   
3            carvao   8.74           4   54.96          em_separacao   
5              cafe  10.16           1   20.16  entregue_com_sucesso   
6            feijao   7.77           3   43.31          em_separacao   

                  cidade estado    pais        cep  frete  \
0                niteroi     RJ  Br

In [94]:

# Função para limpar dados de data e hora
def limpar_data_hora(df, coluna, tipo_coluna):
    print(f"\nLimpando a coluna '{coluna}'")

    if coluna not in df.columns:
        print(f"\nColuna '{coluna}' não encontrada.")
        return df

    # Verificar se existem valores válidos para checar
    serie_str = df[coluna].astype(str).str.strip()
    valores_nao_nulos = serie_str[~serie_str.eq('')]

    if valores_nao_nulos.empty:
        print("\nTodos os valores estão vazios ou nulos.")
        return df

    # Definir máscaras para detectar valores corrompidos
    if tipo_coluna == 'data':
        # Formato correto: "DD/MM/YYYY"
        # Aceita datas como: "01/01/2022", "15/11/2020", "6/9/2020"
        # Agora, se o mês for maior que 12 ou o dia for maior que 31, será considerado corrompido.
        mascara_corrompido = ~serie_str.str.match(r'^(0?[1-9]|[12][0-9]|3[01])/(0?[1-9]|1[0-2])/\d{4}$')
        
        # Verificar se o mês é maior que 12 (esse valor é corrompido)
        corrompido_mes_maior_que_12 = serie_str.str.split('/').str[1].astype(int) > 12
        corrompido_dia_maior_que_31 = serie_str.str.split('/').str[0].astype(int) > 31
        mascara_corrompido |= corrompido_mes_maior_que_12 | corrompido_dia_maior_que_31

    elif tipo_coluna == 'hora':
        # Formato correto: "XX:XX:XX" (hora entre 00 e 23, minuto entre 00 e 59, segundo entre 00 e 59)
        mascara_corrompido = ~serie_str.str.match(r'^(?:[01]?[0-9]|2[0-3]):[0-5][0-9]:[0-5][0-9]$')

    valores_corrompidos = serie_str[mascara_corrompido]

    # Mostrar exemplos de corrompidos
    if not valores_corrompidos.empty:
        print(f"\nValores corrompidos encontrados ({len(valores_corrompidos)}):")
        print(valores_corrompidos.head(10).to_list())
    else:
        print("\nNenhum valor corrompido encontrado.")

    # Limpar os corrompidos
    df_limpo = df[~mascara_corrompido].copy()

    # Mostrar exemplos após a limpeza
    print("\n Exemplo de valores após a limpeza:")
    print(df_limpo[coluna].head(10).to_list())

    return df_limpo


# Aplicando a limpeza nas colunas de 'data' e 'hora'
for coluna, tipo_coluna in zip(['data', 'hora'], ['data', 'hora']):
    df = limpar_data_hora(df, coluna, tipo_coluna)


Limpando a coluna 'data'

Valores corrompidos encontrados (105786):
['3/20/2021', '2/19/2018', '4/21/2018', '9/25/2020', '6/20/2018', '9/15/2021', '4/25/2021', '6/23/2022', '12/25/2020', '3/22/2021']

 Exemplo de valores após a limpeza:
['6/9/2021', '6/4/2022', '10/1/2018', '7/3/2020', '9/7/2019', '12/6/2020', '1/4/2020', '4/10/2022', '6/9/2021', '7/10/2022']

Limpando a coluna 'hora'

Nenhum valor corrompido encontrado.

 Exemplo de valores após a limpeza:
['15:30:28', '8:41:23', '18:36:45', '22:34:21', '15:59:28', '4:03:16', '13:31:19', '14:44:51', '17:38:22', '6:45:58']


In [95]:
print(df.head())

    id_da_compra       data      hora                        cliente  \
2          47484   6/9/2021  15:30:28      felipe_augusto_nery_silva   
3          20809   6/4/2022   8:41:23            levi_ribeiro_amorim   
9          10575  10/1/2018  18:36:45      henrico_da_cunha_teixeira   
12         10784   7/3/2020  22:34:21            mateus_victor_alves   
26          2609   9/7/2019  15:59:28  guilherme_hugo_cazula_pereira   

         produto  valor  quantidade  total                status  \
2   agua_mineral   1.63           9  36.67  pagamento_confirmado   
3         carvao   8.74           4  54.96          em_separacao   
9       sabonete   1.25           2  13.00          em_separacao   
12      manteiga   6.61           8  71.88         em_transporte   
26  oleo_de_soja   3.59          11  59.49          em_separacao   

                   cidade estado    pais        cep  frete  \
2               cabo_frio     RJ  Brasil  28900-000   22.0   
3   campos_dos_goytacazes     RJ  

In [96]:
def tratar_nulos(df):
    # Conversão para numérico e preenchimento
    num_cols = ['VALOR', 'QUANTIDADE', 'FRETE', 'TOTAL']
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
    df[num_cols] = df[num_cols].fillna({
        'VALOR': 0.0,
        'QUANTIDADE': 1,
        'FRETE': 0.0,
        'TOTAL': 0.0
    })

    # Preenchimento e limpeza de textos
    texto_cols = ['CLIENTE', 'PRODUTO', 'STATUS', 'CIDADE', 'ESTADO',
                  'PAIS', 'PAGAMENTO', 'VENDEDOR', 'MARCA']
    df[texto_cols] = df[texto_cols].fillna('desconhecido').apply(lambda x: x.str.strip())

    # Conversão de datas e horas
    df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce').fillna(pd.Timestamp('2000-01-01'))
    df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M:%S', errors='coerce')
    df['HORA'] = df['HORA'].dt.strftime('%H:%M:%S').fillna('00:00:00')

    return df

In [97]:
print(df.head())

    id_da_compra       data      hora                        cliente  \
2          47484   6/9/2021  15:30:28      felipe_augusto_nery_silva   
3          20809   6/4/2022   8:41:23            levi_ribeiro_amorim   
9          10575  10/1/2018  18:36:45      henrico_da_cunha_teixeira   
12         10784   7/3/2020  22:34:21            mateus_victor_alves   
26          2609   9/7/2019  15:59:28  guilherme_hugo_cazula_pereira   

         produto  valor  quantidade  total                status  \
2   agua_mineral   1.63           9  36.67  pagamento_confirmado   
3         carvao   8.74           4  54.96          em_separacao   
9       sabonete   1.25           2  13.00          em_separacao   
12      manteiga   6.61           8  71.88         em_transporte   
26  oleo_de_soja   3.59          11  59.49          em_separacao   

                   cidade estado    pais        cep  frete  \
2               cabo_frio     RJ  Brasil  28900-000   22.0   
3   campos_dos_goytacazes     RJ  

In [98]:
def tratar_cep(df):
    def limpar_cep(cep):
        if pd.isna(cep):
            return '00000000'
        cep_str = ''.join(filter(str.isdigit, str(cep)))
        cep_str = cep_str.zfill(8)
        return cep_str if len(cep_str) == 8 else '00000000'

    df['CEP'] = df['CEP'].apply(limpar_cep)
    return df


In [99]:
print(df.head())

    id_da_compra       data      hora                        cliente  \
2          47484   6/9/2021  15:30:28      felipe_augusto_nery_silva   
3          20809   6/4/2022   8:41:23            levi_ribeiro_amorim   
9          10575  10/1/2018  18:36:45      henrico_da_cunha_teixeira   
12         10784   7/3/2020  22:34:21            mateus_victor_alves   
26          2609   9/7/2019  15:59:28  guilherme_hugo_cazula_pereira   

         produto  valor  quantidade  total                status  \
2   agua_mineral   1.63           9  36.67  pagamento_confirmado   
3         carvao   8.74           4  54.96          em_separacao   
9       sabonete   1.25           2  13.00          em_separacao   
12      manteiga   6.61           8  71.88         em_transporte   
26  oleo_de_soja   3.59          11  59.49          em_separacao   

                   cidade estado    pais        cep  frete  \
2               cabo_frio     RJ  Brasil  28900-000   22.0   
3   campos_dos_goytacazes     RJ  

In [100]:
def validar_e_corrigir(df):
    # Corrigir total onde houver diferença significativa
    total_calculado = df['VALOR'] * df['QUANTIDADE'] + df['FRETE']
    diferenca = (df['TOTAL'] - total_calculado).abs()

    df['TOTAL'] = np.where(diferenca > 0.01, total_calculado, df['TOTAL'])

    # Filtrar valores válidos para cada coluna numérica
    condicoes = (
        (df['QUANTIDADE'] > 0) & (df['QUANTIDADE'] <= 1000) &
        (df['VALOR'] >= 0) & (df['VALOR'] <= 100000) &
        (df['FRETE'] >= 0) & (df['FRETE'] <= 5000)
    )

    return df[condicoes]


In [101]:
print(df.head())

    id_da_compra       data      hora                        cliente  \
2          47484   6/9/2021  15:30:28      felipe_augusto_nery_silva   
3          20809   6/4/2022   8:41:23            levi_ribeiro_amorim   
9          10575  10/1/2018  18:36:45      henrico_da_cunha_teixeira   
12         10784   7/3/2020  22:34:21            mateus_victor_alves   
26          2609   9/7/2019  15:59:28  guilherme_hugo_cazula_pereira   

         produto  valor  quantidade  total                status  \
2   agua_mineral   1.63           9  36.67  pagamento_confirmado   
3         carvao   8.74           4  54.96          em_separacao   
9       sabonete   1.25           2  13.00          em_separacao   
12      manteiga   6.61           8  71.88         em_transporte   
26  oleo_de_soja   3.59          11  59.49          em_separacao   

                   cidade estado    pais        cep  frete  \
2               cabo_frio     RJ  Brasil  28900-000   22.0   
3   campos_dos_goytacazes     RJ  

In [102]:
def validar_estados(df):
    estados_validos = {
        'ac', 'al', 'ap', 'am', 'ba', 'ce', 'df', 'es', 'go', 'ma',
        'mt', 'ms', 'mg', 'pa', 'pb', 'pr', 'pe', 'pi', 'rj', 'rn',
        'rs', 'ro', 'rr', 'sc', 'sp', 'se', 'to', 'desconhecido'
    }
    return df[df['ESTADO'].isin(estados_validos)]


In [103]:
def remover_duplicatas(df):
    return df.drop_duplicates().reset_index(drop=True)


In [104]:
from datetime import datetime

def exportar_resultado(df, output_path, stats_originais):
    try:
        df.to_csv(output_path, index=False, encoding='utf-8')
        removidos = stats_originais['total_registros'] - len(df)
        percentual = (len(df) / stats_originais['total_registros']) * 100

        relatorio = f"""
        ===============================
        RELATÓRIO DE LIMPEZA DE DADOS
        ===============================

        DADOS ORIGINAIS:
        - Total de registros: {stats_originais['total_registros']}
        - Colunas: {', '.join(stats_originais['colunas'])}
        - Valores faltantes: {sum(stats_originais['missing_values'].values())}
        - Registros duplicados: {stats_originais['duplicatas']}

        DADOS FINAIS:
        - Registros válidos: {len(df)}
        - Removidos: {removidos}
        - Percentual mantido: {percentual:.2f}%

        Arquivo salvo em: {output_path}
        Data de execução: {datetime.now():%Y-%m-%d %H:%M:%S}
        """
        print(relatorio)
    except Exception as e:
        raise Exception(f"Erro ao salvar arquivo limpo: {e}")


In [105]:
print(df.head())

    id_da_compra       data      hora                        cliente  \
2          47484   6/9/2021  15:30:28      felipe_augusto_nery_silva   
3          20809   6/4/2022   8:41:23            levi_ribeiro_amorim   
9          10575  10/1/2018  18:36:45      henrico_da_cunha_teixeira   
12         10784   7/3/2020  22:34:21            mateus_victor_alves   
26          2609   9/7/2019  15:59:28  guilherme_hugo_cazula_pereira   

         produto  valor  quantidade  total                status  \
2   agua_mineral   1.63           9  36.67  pagamento_confirmado   
3         carvao   8.74           4  54.96          em_separacao   
9       sabonete   1.25           2  13.00          em_separacao   
12      manteiga   6.61           8  71.88         em_transporte   
26  oleo_de_soja   3.59          11  59.49          em_separacao   

                   cidade estado    pais        cep  frete  \
2               cabo_frio     RJ  Brasil  28900-000   22.0   
3   campos_dos_goytacazes     RJ  

In [106]:
# Exibição final e salvamento do arquivo  
print("\nAmostra final dos dados limpos:")  
print(df.head())  
  
# Salvar arquivo final  
df.to_csv('vendas_modificado_final_completo.csv', index=False)  
print("\nArquivo final salvo como: vendas_modificado_final_completo.csv") 


Amostra final dos dados limpos:
    id_da_compra       data      hora                        cliente  \
2          47484   6/9/2021  15:30:28      felipe_augusto_nery_silva   
3          20809   6/4/2022   8:41:23            levi_ribeiro_amorim   
9          10575  10/1/2018  18:36:45      henrico_da_cunha_teixeira   
12         10784   7/3/2020  22:34:21            mateus_victor_alves   
26          2609   9/7/2019  15:59:28  guilherme_hugo_cazula_pereira   

         produto  valor  quantidade  total                status  \
2   agua_mineral   1.63           9  36.67  pagamento_confirmado   
3         carvao   8.74           4  54.96          em_separacao   
9       sabonete   1.25           2  13.00          em_separacao   
12      manteiga   6.61           8  71.88         em_transporte   
26  oleo_de_soja   3.59          11  59.49          em_separacao   

                   cidade estado    pais        cep  frete  \
2               cabo_frio     RJ  Brasil  28900-000   22.0   
3